In [1]:
# Jupyter magic
%run ../util/dependencies.py
%load_ext sql

Load raw sdr csv

In [ ]:
# sdr_raw query
query_create_table_sdr_raw = """
CREATE TABLE IF NOT EXISTS sdr_raw (
  OperatorControlNumber        text,
  DifficultyDate               text,
  SubmissionDate               text,
  OperatorDesignator           text,
  SubmitterDesignator          text,
  SubmitterTypeCode            text,
  ReceivingRegionCode          text,
  ReceivingDistrictOffice      text,
  SDRType                      text,
  JASCCode                     text,
  NatureOfConditionA           text,
  NatureOfConditionB           text,
  NatureOfConditionC           text,
  PrecautionaryProcedureA      text,
  PrecautionaryProcedureB      text,
  PrecautionaryProcedureC      text,
  PrecautionaryProcedureD      text,
  StageOfOperationCode         text,
  HowDiscoveredCode            text,
  RegistryNNumber              text,
  AircraftMake                 text,
  AircraftModel                text,
  AircraftSerialNumber         text,
  AircraftTotalTime            text,
  AircraftTotalCycles          text,
  EngineMake                   text,
  EngineModel                  text,
  EngineSerialNumber           text,
  EngineTotalTime              text,
  EngineTotalCycles            text,
  PropellerMake                text,
  PropellerModel               text,
  PropellerSerialNumber        text,
  PropellerTotalTime           text,
  PropellerTotalCycles         text,
  PartMake                     text,
  PartName                     text,
  PartNumber                   text,
  PartSerialNumber             text,
  PartCondition                text,
  PartLocation                 text,
  PartTotalTime                text,
  PartTotalCycles              text,
  PartTimeSince                text,
  PartSinceCode                text,
  ComponentMake                text,
  ComponentModel               text,
  ComponentName                text,
  ComponentPartNumber          text,
  ComponentSerialNumber        text,
  ComponentLocation            text,
  ComponentTotalTime           text,
  ComponentTotalCycles         text,
  ComponentTimeSince           text,
  ComponentSinceCode           text,
  FuselageStationFrom          text,
  FuselageStationTo            text,
  StringerFrom                 text,
  StringerFromSide             text,
  StringerTo                   text,
  StringerToSide               text,
  WingStationFrom              text,
  WingStationFromSide          text,
  WingStationTo                text,
  WingStationToSide            text,
  ButtLineFrom                 text,
  ButtLineFromSide             text,
  ButtLineTo                   text,
  ButtLineToSide               text,
  WaterLineFrom                text,
  WaterLineTo                  text,
  CrackLength                  text,
  NumberOfCracks               text,
  CorrosionLevel               text,
  StructuralOther              text,
  Discrepancy                  text
);
"""

# 'vectra' database connection
db_connection = psycopg2.connect(dbname='vectra',
                                 host='localhost',
                                 user='robb',
                                 password='thek3yisK#')
cur = db_connection.cursor()
cur.execute(query_create_table_sdr_raw)

# copy contents to sdr_raw table
with open('../data/02_csv/SDR_COMPOSITE_EXPORT.csv', 'r') as f:
    cur.copy_expert("""
        COPY sdr_raw
        FROM STDIN
        WITH CSV HEADER
    """, f)

# commit and close the connection
db_connection.commit()
cur.close()
db_connection.close()

Check sdr_raw

In [ ]:
-- Lets test first to ensure that sdr data was properly transfered over to the 'sdr_raw' table . . .
SELECT *
FROM sdr_raw
LIMIT 10;

-- Ensure csv size . . .
SELECT count(*)
FROM sdr_raw;

-- Alright, this checks out!!

Lets start normalizing the database, by transferring contents of sdr_raw into our other data tables (aircraft, ata_chapter, sdr_event, &)

In [ ]:
-- ================================================================
-- Populating the Master Tables (aircraft, ata_chapter, & operator)
-- ================================================================

-- ata_chapter
INSERT INTO ata_chapter(jasc_code)
SELECT DISTINCT JASCCODE
FROM raw_sdr
WHERE JASCCode IS NOT NULL
ON CONFLICT (jasc_code) DO NOTHING;

-- operator
INSERT INTO operator (operator_code)
SELECT DISTINCT OperatorDesignator
FROM raw_sdr
WHERE OperatorDesignator IS NOT NULL
ON CONFLICT (operator_code) DO NOTHING;

-- aircraft
INSERT INTO aircraft (
    registry_n_number,
    aircraft_make,
    aircraft_model,
    aircraft_serial,
    aircraft_total_time,
    aircraft_total_cycles)
SELECT DISTINCT RegistryNNumber,
    AircraftMake,
    AircraftModel,
    AircraftSerialNumber,
    AircraftTotalTime,
    AircraftTotalCycles
FROM sdr_raw
WHERE RegistryNNumber IS NOT NULL
ON CONFLICT (registry_n_number) DO UPDATE
    SET aircraft_make = EXCLUDED.aircraft_make,
    aircraft_model = EXCLUDED.aircraft_model,
    aircraft_serial = EXCLUDED.aircraft_serial,
    aircraft_total_time = EXCLUDED.aircraft_total_time,
    aircraft_total_cycles = EXCLUDED.aircraft_total_cycles;